# Setup
#### arguments, packages, seeds, functions

In [ ]:
print("inputting arguments...")

# training arguments
N_vec = [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000]  # Brian liked: [100, 500, 1000, 2000, 3000, 4000, 5000, 6000] 
PROP_VAL = 0.5 # 0.2

# normalizing flow arguments
NUMBER_LAYERS = 4
LEARNING_RATE = 0.0005
NUM_EPOCHS = 100
EARLY_STOPPING = True
BATCH_SIZE = 100

# true value of parameters
LAMBDA = .4
I0 = 20
PSI = 7 # 7 is best


print("finished inputting arguments!")


inputting arguments...
finished inputting arguments!


In [37]:
PSI

1

In [26]:
print("importing packages...")

# original ones from the sparse linear regression example
import numpy as np
import bayesflow as bf
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# additional ones from the SIR tutorial
import datetime
from functools import partial
import pandas as pd

# used in the "convert_params" function
from scipy.stats import nbinom

import tensorflow as tf

RNG = np.random.default_rng(seed=2023)


print("finished importing packages!")

importing packages...
finished importing packages!


In [27]:
# loading in the PCA matrix
loading_matrix = np.loadtxt("./loading_matrix.csv", delimiter=",", dtype=str)
loading_matrix = loading_matrix[1:,1:]
loading_matrix = np.array(loading_matrix, dtype = float)

In [28]:
# loading in the Y-scaling info
logY_mn = np.loadtxt("./logY_mn.csv", delimiter=",", dtype=str)
logY_mn = logY_mn[1:,1:]
logY_mn = np.array(logY_mn, dtype = float)
logY_sd = np.loadtxt("./logY_sd.csv", delimiter=",", dtype=str)
logY_sd = logY_sd[1:,1:]
logY_sd = np.array(logY_sd, dtype = float)
Z_mn = np.loadtxt("./Z_mn.csv", delimiter=",", dtype=str)
Z_mn = Z_mn[1:,1:]
Z_mn = np.array(Z_mn, dtype = float)
Z_sd = np.loadtxt("./Z_sd.csv", delimiter=",", dtype=str)
Z_sd = Z_sd[1:,1:]
Z_sd = np.array(Z_sd, dtype = float)

In [29]:
np.squeeze(logY_mn).shape

(14,)

In [30]:
print("defining functions...")

# prior for the nonspatial SIR problem
def model_prior():
    """Generates a random draw from the joint prior."""

    # taken from the Bayesflow website
    lambd = RNG.lognormal(mean=np.log(0.4), sigma=0.5)
    # mu = RNG.lognormal(mean=np.log(1 / 8), sigma=0.2)
    # D = RNG.lognormal(mean=np.log(8), sigma=0.2)
    I0 = RNG.gamma(shape=2, scale=20)
    psi = RNG.exponential(5)
    # return np.array([lambd, mu, D, I0, psi])
    # return np.array([lambd, mu, I0])
    return np.array([lambd, I0, psi])

# the data of the nonspatial SIR model are negative binomially distributed... these convert the parameters of interest into r and p...
def convert_params(mu, phi):
    """Helper function to convert mean/dispersion parameterization of a negative binomial to N and p,
    as expected by numpy's negative_binomial.

    See https://en.wikipedia.org/wiki/Negative_binomial_distribution#Alternative_formulations
    """

    r = phi
    var = mu + 1 / r * mu**2
    p = (var - mu) / var
    return r, 1 - p

# generate data from the nonspatial SIR model given parameters
def stationary_SIR(params, N, T, eps=1e-5):
    """Performs a forward simulation from the stationary SIR model given a random draw from the prior."""

    # Extract parameters and round I0 and D
    # lambd, mu, D, I0, psi = params
    # lambd, mu, I0 = params
    lambd, I0, psi = params
    D = 0
    mu = .1
    # psi = 1
    I0 = np.ceil(I0)
    D = int(round(D))

    # Initial conditions
    S, I, R = [N - I0], [I0], [0]

    # Reported new cases
    C = [I0]

    # Simulate T-1 timesteps
    for t in range(1, T + D):
        # Calculate new cases
        I_new = lambd * (I[-1] * S[-1] / N)

        # SIR equations
        S_t = S[-1] - I_new
        I_t = np.clip(I[-1] + I_new - mu * I[-1], 0.0, N)
        R_t = np.clip(R[-1] + mu * I[-1], 0.0, N)

        # Track
        S.append(S_t)
        I.append(I_t)
        R.append(R_t)
        C.append(I_new)

    reparam = convert_params(np.clip(np.array(C[D:]), 0, N) + eps, psi)
    C_obs = RNG.negative_binomial(reparam[0], reparam[1])
    # print("C_obs.shape")
    # print(C_obs.shape)
    C_obs = np.log(C_obs + 1)
    # print("C_obs.shape")
    # print(C_obs.shape)
    C_obs = (C_obs - np.squeeze(logY_mn)) / np.squeeze(logY_sd)
    # print("C_obs.shape")
    # print(C_obs.shape)
    score = np.dot(C_obs,loading_matrix)
    # print("score.shape")
    # print(score.shape)
    score = (score - np.squeeze(Z_mn)) / np.squeeze(Z_sd)
    return score[:, np.newaxis]

# use this if we are using the historical SIR cases (we are not)
def load_data():
    """Helper function to load cumulative cases and transform them to new cases."""

    confirmed_cases_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
    confirmed_cases = pd.read_csv(confirmed_cases_url, sep=",")

    date_data_begin = datetime.date(2020, 3, 1)
    date_data_end = datetime.date(2020, 3, 15)
    format_date = lambda date_py: f"{date_py.month}/{date_py.day}/{str(date_py.year)[2:4]}"
    date_formatted_begin = format_date(date_data_begin)
    date_formatted_end = format_date(date_data_end)

    cases_obs = np.array(
        confirmed_cases.loc[confirmed_cases["Country/Region"] == "Germany", date_formatted_begin:date_formatted_end]
    )[0]
    new_cases_obs = np.diff(cases_obs)
    return new_cases_obs


# plots the posterior predictive distribution as a measure of fit
def plot_ppc(config, post_samples, logscale=True, color="#8f2727", dummy=True, figsize=(12, 6), font_size=18):
    """
    Helper function to perform some plotting of the posterior predictive.
    """
    # Plot settings
    plt.rcParams["font.size"] = font_size

    # Remove parameters < 0
    samples = post_samples[np.sum(post_samples < 0, axis=1) == 0]

    f, ax = plt.subplots(1, 1, figsize=figsize)

    # Re-simulations
    sims = []
    for i in range(samples.shape[0]):
        # Note - simulator returns 2D arrays of shape (T, 1), so we remove trailing dim
        sim_cases = stationary_SIR(samples[i], config["N"], config["T"])[:, 0]
        sims.append(sim_cases)
    sims = np.array(sims)

    # Compute quantiles for each t = 1,...,T
    qs_50 = np.quantile(sims, q=[0.25, 0.75], axis=0)
    qs_90 = np.quantile(sims, q=[0.05, 0.95], axis=0)
    qs_95 = np.quantile(sims, q=[0.025, 0.975], axis=0)

    # Plot median predictions and observed data
    ax.plot(np.median(sims, axis=0), label="Median predicted cases", color=color)
    ax.plot(config["obs_data"], marker="o", label="Reported cases", color="black", linestyle="dashed", alpha=0.8)

    # Add compatibility intervals (also called credible intervals)
    ax.fill_between(range(config["T"]), qs_50[0], qs_50[1], color=color, alpha=0.5, label="50% CI")
    ax.fill_between(range(config["T"]), qs_90[0], qs_90[1], color=color, alpha=0.3, label="90% CI")
    ax.fill_between(range(config["T"]), qs_95[0], qs_95[1], color=color, alpha=0.1, label="95% CI")

    # Grid and schmuck
    ax.grid(color="grey", linestyle="-", linewidth=0.25, alpha=0.5)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.set_xlabel("Days since pandemic onset")
    ax.set_ylabel("Number of cases")
    ax.minorticks_off()
    if logscale:
        ax.set_yscale("log")
    ax.legend(fontsize=font_size)
    return f

# get quantiles of a distribution (used to get the median and credible intervals)
def get_quantiles(x):
    return(np.array([np.quantile(x, .05), np.quantile(x, .50), np.quantile(x, .95)]))


# CONFIGURATOR WHEN NO SUMMARY NETWORK IS PROVIDED
def no_summary_net(simulation_outputs):
    prior_draws = simulation_outputs["prior_draws"].astype(np.float32)

    # # standardize prior_draws
    # prior_means = np.mean(prior_draws, axis=0)
    # prior_stds = np.std(prior_draws, axis=0)
    std_prior_draws = (prior_draws - prior_means) / prior_stds # seems to work... we got good results

    # transform sim_data
    # logdata = np.log1p(simulation_outputs["sim_data"] + 1).astype(np.float32)
    data = simulation_outputs["sim_data"].astype(np.float32)
    
    configured_outputs = {
        # "summary_conditions": None,
        "direct_conditions":data[:,:,0], # to match the format for inference_networks
        "parameters": std_prior_draws, # the [:,p:] is for picking out the specific covariates
    }
    return configured_outputs

# # CONFIGURATOR WHEN A SUMMARY NETWORK IS PROVIDED
# def vanilla_configurator(simulation_outputs):
#     prior_draws = simulation_outputs["prior_draws"].astype(np.float32)

#     # standardize prior_draws
#     prior_means = np.mean(prior_draws, axis=0)
#     prior_stds = np.std(prior_draws, axis=0)
#     std_prior_draws = (prior_draws - prior_means) / prior_stds # seems to work... we got good results

#     # transform sim_data
#     # logdata = np.log1p(simulation_outputs["sim_data"] + 1).astype(np.float32)
#     logdata = simulation_outputs["sim_data"]

#     configured_outputs = {
#         "summary_conditions": logdata, # already matches the format for summary_networks
#         "direct_conditions":None,
#         "parameters": std_prior_draws,
#     }
#     return configured_outputs

# # CONFIGURATOR FOR THE Bayesflow SIR TUTORIAL (unused)
# def configure_input(forward_dict):
#     """Function to configure the simulated quantities (i.e., simulator outputs)
#     into a neural network-friendly (BayesFlow) format.
#     """

#     # Prepare placeholder dict
#     out_dict = {}

#     # Convert data to logscale
#     logdata = np.log1p(forward_dict["sim_data"]).astype(np.float32)

#     # Extract prior draws and z-standardize with previously computed means
#     params = forward_dict["prior_draws"].astype(np.float32)
    
#     # Calculate column means
#     prior_means = np.mean(params, axis=0)
    
#     # Calculate column standard deviations
#     prior_stds = np.std(params, axis=0)

#     # standardized the parameter draws
#     params = (params - prior_means) / prior_stds # seems to work... we got good results

#     # Remove a batch if it contains nan, inf or -inf
#     idx_keep = np.all(np.isfinite(logdata), axis=(1, 2))
#     if not np.all(idx_keep):
#         print("Invalid value encountered...removing from batch")
#     # Add to keys
#     out_dict["summary_conditions"] = logdata[idx_keep]
#     out_dict["parameters"] = params[idx_keep]

#     return out_dict

# print("finished defining functions!")


defining functions...


# Simulation Study Across Training Sizes

In [31]:
# define the nonspatial SIR model

# defining the prior object, with names of the parameters
prior = bf.simulation.Prior(prior_fun=model_prior, param_names=[r"$\lambda$", r"$I_0$", r"$\psi$"])
prior_means, prior_stds = prior.estimate_means_and_stds()

# defining the simulation object, with some fixed parameters
SIR_num_timepoints_T = 14
SIR_population_N = 83e6
simulator = bf.simulation.Simulator(simulator_fun=partial(stationary_SIR, T=SIR_num_timepoints_T, N=SIR_population_N))

# pairing the prior and simulator (now simulations can be drawn)
model = bf.simulation.GenerativeModel(prior, simulator)

# note that output comes out after the GenerativeModel initialization, which tells you the dimensions of everything

INFO:root:Performing 2 pilot runs with the anonymous model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 3)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 7, 1)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional simulation batchable context provided.


In [32]:
# define the normalizing flow

# # defining the summary network (we aren't using this)
# summary_net = bf.networks.SequenceNetwork()

# DEFINING THE INFERENCE NETWORK -- part 2 of the Normalizing Flow
mydesign = "affine" # choices: "affine" (harder problems, more general), "spline" (works well for low-dimensional problems)
permute = "fixed" # choices: "fixed" (highly recommended, but permutes them), None (no permutation), "learnable"

inference_net = bf.networks.InvertibleNetwork(
    num_params=3,
    num_coupling_layers=NUMBER_LAYERS
    # coupling_design=mydesign,
    # permutation=permute
)

In [33]:
# simulation study setup

# true values--copied from VaNBayes simulation study
true_lambda = LAMBDA
true_I0 = I0
true_psi = PSI
true_parameters = np.array([true_lambda, true_I0, true_psi])
nparams = len(true_parameters)

# importing the simulation study datasets from the VaNBayes simulation study
sim_study_datasets = np.loadtxt("./sim_study_datasets.csv", delimiter=",", dtype=str)
sim_study_datasets = sim_study_datasets[1:,1:]
sim_study_datasets = np.array(sim_study_datasets, dtype = float)
print(sim_study_datasets.shape) # nsims x length_of_each_simulated_dataset
nsims = sim_study_datasets.shape[0]

posterior_resolution = 10000 # how we'll take the MC estimates of our resulting posteriors

# sample sizes the pick from 
# N_vec = [1250,2500,6250,12500,25000,62500,125000]
# training sizes are: [  1000,   2000,   5000,  10000,  20000,  50000, 100000]

(100, 7)


In [34]:
# perform the Bayesflow simulation study

# # OLD
# parameters_captured = np.array([0,0,0], np.uint8)
# parameter_ADs = np.zeros((nsims, nparams))
# posterior_medians = np.zeros((nsims, nparams))

parameters_captured = np.zeros((nparams, len(N_vec)))
parameter_ADs = np.zeros((nsims, nparams, len(N_vec)))
posterior_medians = np.zeros((nsims, nparams, len(N_vec)))


for k in range(len(N_vec)):

    # define the sample size
    N_total = N_vec[k]

    # train Bayesflow with this sample size...

    # define a new trainer and amortizer (so each Bayesflow architecture is trained separately)
    amortizer = bf.amortizers.AmortizedPosterior(inference_net)
    trainer = bf.trainers.Trainer(amortizer=amortizer, 
                                generative_model=model, 
                                configurator=no_summary_net, 
                                default_lr=LEARNING_RATE)
    
    # data split
    N_tr = round((1-PROP_VAL)*N_total)
    N_val = round(PROP_VAL*N_total)
    offline_data = model(N_tr) # same number as VaNBayes

    # train the neural network
    history = trainer.train_offline(offline_data, 
                                    epochs=NUM_EPOCHS, 
                                    early_stopping=EARLY_STOPPING, # potential to stop early if not improving...
                                    batch_size=BATCH_SIZE, 
                                    validation_sims=N_val)
    
    # once trained, run the simulation study 
    for i in range(nsims): # for each of the 100 datasets...

        # # format the simulation study dataset to obtain posterior draws (OLD)
        # obs_data = np.log1p(sim_study_datasets[i,:])[np.newaxis, :, np.newaxis].astype(np.float32)
        # post_samples = amortizer.sample({"summary_conditions": obs_data}, posterior_resolution)

        # Obtain posterior draws (NEW)
        normalizing_flow_inputs = trainer.configurator({"prior_draws": true_parameters[np.newaxis,:], "sim_data":sim_study_datasets[i,:][np.newaxis, :, np.newaxis]})
        post_samples = amortizer.sample(normalizing_flow_inputs, n_samples=posterior_resolution) # draw n_samples from the posterior FOR EACH generated dataset (prior and sim draws)
        
        # Undo standardization to get parameters on their original (unstandardized) scales
        post_samples = prior_means + post_samples * prior_stds

        # get the posterior quantiles
        median_and_90credible_interval = np.apply_along_axis(get_quantiles, 0, post_samples)

        for j in range(nparams):

            # coverage capturing
            if (median_and_90credible_interval[0,j] < true_parameters[j])&(true_parameters[j] < median_and_90credible_interval[2,j]):
                parameters_captured[j,k] = parameters_captured[j,k] + 1

            # absolute deviation
            parameter_ADs[i,j,k] = np.abs(median_and_90credible_interval[1,j] - true_parameters[j])

            # save the medians to send to R
            posterior_medians[i,j,k] = median_and_90credible_interval[1,j]

INFO:root:Performing a consistency check with provided components...
INFO:root:Done.
INFO:root:Generated 250 simulations for validation.


Training epoch 1:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:root:Validation, Epoch: 1, Loss: 4.268


Training epoch 2:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:root:Validation, Epoch: 2, Loss: 4.222


...

INFO:root:Validation, Epoch: 18, Loss: 0.416


Training epoch 19:   0%|          | 0/30 [00:00<?, ?it/s]

INFO:root:Validation, Epoch: 19, Loss: 0.439


Training epoch 20:   0%|          | 0/30 [00:00<?, ?it/s]

INFO:root:Validation, Epoch: 20, Loss: 0.426
INFO:root:Early stopping triggered.


In [35]:
# print some average results for a sanity check:
print("number of sample sizes: ")
len(N_vec)
parameter_coverage = parameters_captured/nsims
parameter_MAD = np.apply_along_axis(np.mean, 0, parameter_ADs)
parameter_SDAD = np.apply_along_axis(np.std, 0, parameter_ADs)
print("parameter_coverage")
print(parameter_coverage.shape)
print(parameter_coverage)
print("parameter_MAD")
print(parameter_MAD.shape)
print(parameter_MAD)
print("parameter_SDAD")
print(parameter_SDAD.shape)
print(parameter_SDAD)

number of sample sizes: 
parameter_coverage
(3, 12)
[[0.98 0.85 0.94 0.92 0.93 0.94 0.88 0.91 0.89 0.87 0.88 0.88]
 [0.95 0.96 0.93 0.9  0.93 0.98 0.97 0.93 0.93 0.91 0.93 0.92]
 [1.   0.96 0.95 0.9  0.91 0.93 0.86 0.91 0.94 0.93 0.88 0.9 ]]
parameter_MAD
(3, 12)
[[ 0.05678063  0.0549241   0.05211161  0.05018132  0.04780396  0.04388927
   0.04296795  0.04452662  0.04719498  0.04681769  0.0479033   0.04938593]
 [12.65347018  9.70607938  8.32410468  8.30810091  8.31284059  7.6457084
   8.15694054  9.01204212  7.93365805  8.56802239  8.52909941  7.95100089]
 [ 1.71430312  0.77405457  0.84132197  0.87206148  0.82558611  0.86472381
   0.86512512  0.88592169  0.8489436   0.83904391  0.91413582  0.79256404]]
parameter_SDAD
(3, 12)
[[0.04951644 0.04526734 0.04275781 0.0421028  0.03962932 0.03914912
  0.04163916 0.03971002 0.03951087 0.04037906 0.04079754 0.03919665]
 [9.4982256  8.59787411 7.24553807 6.95106058 6.75599264 6.10908554
  6.42592284 6.94761974 6.21841069 6.65373994 6.93993176 5.84

In [36]:
# save the Bayesflow results-- to be used in compile_results.R
np.savetxt("./bayesflow_parameter_coverage.csv", parameter_coverage, delimiter=",")
np.savetxt("./bayesflow_parameter_MAD.csv", parameter_MAD, delimiter=",")
np.savetxt("./bayesflow_parameter_SDAD.csv", parameter_SDAD, delimiter=",")

for k in range(len(N_vec)):
    N_total = N_vec[k]
    saved_filename = "./bayesflow_posterior_medians_N_"+str(N_total)+".csv"
    np.savetxt(saved_filename, posterior_medians[:,:,k], delimiter=",")